In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime 
import locale                                                           
import re
from tqdm import tqdm_notebook,tqdm,notebook
import matplotlib.pylab as plt
import json
from datetime import datetime
import pickle

plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
locale.setlocale(locale.LC_ALL, 'ko_KR.UTF-8') 

In [ ]:
performance_data = pd.read_csv('../data/base_data_concat.csv')
weather = pd.read_csv("../data/2019_weather_data.csv",index_col=False)
weather.drop(['Unnamed: 0'],axis=1,inplace=True)
money = pd.read_excel('../data/money2.xls')
weather_2020 = pd.read_csv("../data/20206_weather_data.csv",index_col=False)
money_2020 = pd.read_excel('../data/20206_주가지수.xls')

In [ ]:
display(performance_data.head())
display(weather.head())
display(money.head())
display(weather_2020.head())
display(money_2020.head())

### **날씨 붙이기**

In [ ]:
weather.일시 = weather.일시.map(str)
weather['date'] = weather.일시.apply(lambda x: int(x[:8]))
weather['hour'] = weather.일시.apply(lambda x: int(x[8:10]))
weather['minute'] = weather.일시.apply(lambda x: int(x[10:]))

weather_2020['date'] = weather_2020.일시.apply(lambda x: int(re.sub('-','',x.split(' ')[0])))
weather_2020['hour'] = weather_2020.일시.apply(lambda x: int(x.split(' ')[1].split(':')[0]))
weather_2020['minute'] = weather_2020.일시.apply(lambda x: int(x.split(' ')[1].split(':')[1]))

In [ ]:
merge_data = performance_data.drop_duplicates(['date','hour','minute']).reset_index(drop=True)
merge_data.방송일시 = merge_data.방송일시.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
merge_data

In [ ]:
merge_data = performance_data.drop_duplicates(['date','hour','minute']).reset_index(drop=True)
merge_data.방송일시 = merge_data.방송일시.apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
merge_data['subMin'] = merge_data.loc[1:,'방송일시'].reset_index(drop=True) - merge_data.loc[:22180,'방송일시']
merge_data['subMin'] = merge_data['subMin'].apply(lambda x: x.seconds/60)
merge_data = merge_data.iloc[:-1,:] # 2020년 데이터, 시청률 없음.

In [ ]:
weather = weather.iloc[:,1:]
weather_2020 = weather_2020.iloc[:,1:]

In [ ]:
weather = pd.concat([weather,weather_2020]).reset_index(drop=True)

In [ ]:
for col in weather.columns[:4]:
    d = []
    for i in tqdm(range(len(merge_data))):
        date = merge_data.date[i]
        hour = merge_data.hour[i]
        minute = merge_data.minute[i]
        try:
            idx = weather[(weather['date'] == date) & (weather['hour'] == hour) & (weather['minute'] == minute)].index[0]
            a = weather.loc[idx:int(idx+merge_data.subMin[i]),col]
            d.append(a.mean())
        except:
            d.append(np.nan)
    merge_data[col] = d

In [ ]:
merge_data['date'] = merge_data['date'].map(str)
performance_data['date'] = performance_data['date'].map(str)

### **주가 붙이기**

In [ ]:
money = pd.concat([money,money_2020]).reset_index(drop=True)

In [ ]:
money['date'] = money['일자'].apply(lambda x: re.sub('/','',x))
del money['일자']

In [ ]:
merge_data = pd.merge(merge_data,money,on='date',how='left')

### **데이터 통합 후 저장**

In [ ]:
merge_data['방송일시'] = merge_data['방송일시'].map(str)
merge_data = merge_data[['방송일시'] + list(merge_data.columns[19:])]
performance_data = performance_data.merge(merge_data,on='방송일시',how='left')
performance_data.head()

In [ ]:
performance_data.loc[0,money.columns[:-1]] = money.iloc[18,:-1]
performance_data = performance_data.fillna(method='ffill')

In [ ]:
performance_data.info()

In [ ]:
for i in ['현재지수']+list(performance_data.columns[28:]):
    performance_data[i] = performance_data[i].apply(lambda x: re.sub(',','',x))
    performance_data[i] = performance_data[i].map(float)

In [ ]:
with open('../data/merged_data_concat.pickle', 'wb') as f:
    pickle.dump(performance_data, f)